# Fallstudie Tablet GmbH

# Problemstellung

- Warum war die wirtschaftlich Entwicklung in den letzten beiden Jahren negativ? 
- Welche Faktoren beeinflussten die Situation?
- Wie kann die wirtschaftliche Situation verbessert werden?

# Hypothesen & Fragestellungen (bisher):
- Veränderung der Nachfrage hin zu Tablet Pro & Tablet Mini und weg von Tablet Basic (Silas)
    - Customer Rating als möglicher Indikator (Daniel)
- Downtime als Grund für den geringeren Umsatz in Online-Vertriebskanälen (Vincenzo)
- Sind bestimmte Regionen abhängig von bestimmten Vertriebskanälen/Produkten? (Ali)
- Was verursachte den Umsatzrückgang zum Jahreswechsel? (Ali)

# 0) Allgemeine Einstellungen

In [ ]:
# Importieren der benötigten Bibliotheken
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np
import statsmodels.formula.api as smf

In [ ]:
# Zahlen in Dataframes ohne wissenschaftliche Notation ausgeben (zwei Nachkommastellen und Tausendertrennzeichen)
pd.options.display.float_format = '{:,.2f}'.format

# 1) Einlesen

In [ ]:
df_crm = pd.read_csv("./data_csv/CRM_data.csv")
df_downtime = pd.read_csv("./data_csv/Downtime.csv")
df_preise = pd.read_csv("./data_csv/Preise.csv")

df_a_online = pd.read_csv("./data_csv/RegionA_online.csv")
df_a_phone = pd.read_csv("./data_csv/RegionA_phone.csv")
df_a_store = pd.read_csv("./data_csv/RegionA_store.csv")

df_b_online = pd.read_csv("./data_csv/RegionB_online.csv")
df_b_phone = pd.read_csv("./data_csv/RegionB_phone.csv")
df_b_store = pd.read_csv("./data_csv/RegionB_store.csv")

df_c_online = pd.read_csv("./data_csv/RegionC_online.csv")
df_c_phone = pd.read_csv("./data_csv/RegionC_phone.csv")
df_c_store = pd.read_csv("./data_csv/RegionC_store.csv")

df_d_online = pd.read_csv("./data_csv/RegionD_online.csv")
df_d_phone = pd.read_csv("./data_csv/RegionD_phone.csv")
df_d_store = pd.read_csv("./data_csv/RegionD_store.csv")

# 2) Aufbereiten

## CRM-Daten:

In [ ]:
df_crm.head()

In [ ]:
df_crm.info()

In [ ]:
# Spalten auf Englisch umbenennen
df_crm = df_crm.rename(columns={
    'vorname': 'first_name',
    'nachname': 'last_name',
})

df_crm.head()

## Preisdaten:

In [ ]:
df_preise

In [ ]:
# Umbenennen der Spalte "Jahr" in "year"
df_preise = df_preise.rename(columns={"Jahr": "year",
                                      "preis": "price"})

# Entfernen von Anfuehrungszeichen
df_preise['year'] = df_preise['year'].apply(lambda x: x.strip('"')) 

# Korrigieren der falschen Jahreszahlen
df_preise['year'] = df_preise['year'].replace({'20022': '2022'})  

# Umwandeln der Spalte "jahr" in Datumsformat
df_preise['year'] = pd.to_datetime(df_preise['year'], format='%Y')

# Spalte "jahr" als Jahreszahl extrahieren
df_preise['year'] = df_preise['year'].dt.year

# Werte "Tablet Basics" in "Tablet Basic" ändern
df_preise['product'] = df_preise['product'].replace({'Tablet Basics': 'Tablet Basic'})

df_preise

## Downtime-Daten:

In [ ]:
df_downtime

In [ ]:
# Erste Spalte in "year" umbenennen
df_downtime = df_downtime.rename(columns={"Unnamed: 0": "year"})

# Spalte "year" in Datumsformat umwandeln
df_downtime['year'] = pd.to_datetime(df_downtime['year'], format='%Y')

# Spalte "year" als Jahreszahl extrahieren
df_downtime['year'] = df_downtime['year'].dt.year

# Umbenennen der Spalten "Region A" bis "Region D" in "region_a" bis "region_d"
df_downtime = df_downtime.rename(columns={"Region A": "region_a", 
                                          "Region B": "region_b", 
                                          "Region C": "region_c", 
                                          "Region D": "region_d"})

df_downtime

In [ ]:
# df_downtime umformen: Spalten sind die beiden Jahre und die Zeilen die Regionen
df_downtime = df_downtime.melt(id_vars=['year'], var_name='region', value_name='downtime')

# Werte in der Spalte "region" in "A" bis "D" umbenennen
df_downtime['region'] = df_downtime['region'].replace({'region_a': 'A', 
                                                       'region_b': 'B', 
                                                       'region_c': 'C', 
                                                       'region_d': 'D'})

df_downtime

## Transaktionsdaten:

In [ ]:
# Hinzufuegen der Spalten "region" und "channel" mit den entsprechenden Werten
df_a_online = df_a_online.assign(region='A', channel='online')
df_a_phone = df_a_phone.assign(region='A', channel='phone')
df_a_store = df_a_store.assign(region='A', channel='store')

df_b_online = df_b_online.assign(region='B', channel='online')
df_b_phone = df_b_phone.assign(region='B', channel='phone')
df_b_store = df_b_store.assign(region='B', channel='store')

df_c_online = df_c_online.assign(region='C', channel='online')
df_c_phone = df_c_phone.assign(region='C', channel='phone')
df_c_store = df_c_store.assign(region='C', channel='store')

df_d_online = df_d_online.assign(region='D', channel='online')
df_d_phone = df_d_phone.assign(region='D', channel='phone')
df_d_store = df_d_store.assign(region='D', channel='store')

In [ ]:
# Umbenennen der Spalte "PRODUKT" in "product" in df_b_phone
df_b_phone = df_b_phone.rename(columns={"PRODUKT": "product"})

In [ ]:
# Alle Regionen in einen Dataframe zusammenfassen
df_sales = pd.concat([df_a_online, df_a_phone, df_a_store,
                      df_b_online, df_b_phone, df_b_store, 
                      df_c_online, df_c_phone, df_c_store, 
                      df_d_online, df_d_phone, df_d_store])

In [ ]:
# Umbenennen der Spalten "Tablet Basic", "Tablet Mini" und "Tablet Pro", "kosten_basic", "kosten_mini" und "kosten_pro"
df_sales = df_sales.rename(columns={
    "Tablet Basic": "tablet_basic",
    "Tablet Mini": "tablet_mini",
    "Tablet Pro": "tablet_pro",
    "kosten_basic": "cost_basic",
    "kosten_mini": "cost_mini",
    "kosten_pro": "cost_pro"
})

In [ ]:
# Umwandeln der Spalte "jahr" in Datumsformat
df_sales["date"] = pd.to_datetime(df_sales["date"])

In [ ]:
# NaN in den Spalten cost_basic, cost_mini und cost_pro durch 0 ersetzen
df_sales['cost_basic'] = df_sales['cost_basic'].fillna(0)
df_sales['cost_mini'] = df_sales['cost_mini'].fillna(0)
df_sales['cost_pro'] = df_sales['cost_pro'].fillna(0)

In [ ]:
# NaN in "discount" durch 1 ersetzen
df_sales['discount'] = df_sales['discount'].fillna(1)

# Discount tatsächlichen Wert berechnen
df_sales['discount'] = 1 - df_sales['discount']

In [ ]:
# Zeige Werte der Spalte "product" an
df_sales['product'].unique()

In [ ]:
# Werte "Tablet Basic..." in "Tablet Basic" aendern
df_sales['product'] = df_sales['product'].replace({'Tablet Basic...': 'Tablet Basic'})

In [ ]:
df_sales.info()

In [ ]:
df_sales

# 3) Transformieren

In [ ]:
# df_sales und df_crm zusammenfuegen
df_sales = pd.merge(df_sales, df_crm, left_on='transaction_id', right_on='id', how='inner')

# Spalte "id" entfernen
df_sales = df_sales.drop(columns=['id'])

# Neue Spalten "year", "month" und "week" erstellen
df_sales = df_sales.assign(year=df_sales['date'].dt.year,
                           month=df_sales['date'].dt.month,
                           week=df_sales['date'].dt.isocalendar().week)

# Neue Spalte "price" erstellen, basierend auf den Werten in der Spalte "price" in df_preise
df_sales = df_sales.merge(df_preise, on=["year", "product"], how="left")

# Neue Spalte "cost" erstellen
df_sales['cost'] = df_sales["cost_basic"] + df_sales["cost_mini"] + df_sales["cost_pro"]

# Neue Spalte "cost_total" erstellen
df_sales['cost_total'] = df_sales["cost"] * df_sales["amount"]

# Neue Spalte "revenue" erstellen
df_sales['revenue'] = df_sales['price'] * df_sales['amount'] * (1 - df_sales['discount'])

# Neue Spalte "profit_per_piece" erstellen
df_sales["profit_per_piece"] = (df_sales["price"] * (1 - df_sales["discount"])) - df_sales["cost"]

# Neue Spalte "profit_total" erstellen
df_sales["profit_total"] = df_sales["profit_per_piece"] * df_sales["amount"]

# Alle Spalten anzeigen lassen
pd.set_option('display.max_columns', None)
df_sales

In [ ]:
# Drop columns "first_name" and "last_name", "iban", "email"
df_sales = df_sales.drop(columns=['first_name', 'last_name', 'iban', 'email'])
df_sales

In [ ]:
# Sortieren nach Datum
df_sales.sort_values(by=['transaction_id'], inplace=True)

# Reset der Index-Spalte
df_sales.reset_index(drop=True, inplace=True)

In [ ]:
df_sales

In [ ]:
df_sales.info()

In [ ]:
# summe der kosten nach produkt
df_sales.groupby(['product'])['cost_total'].sum()

# 4) Analyse der allgemeinen wirtschaftlichen Entwicklung

## Entwicklung der Kennzahlen Verkaufsmenge, Umsatz, Kosten, Gewinn und Umsatzrendite

In [ ]:
# df_sales nach Jahr und Monat gruppieren und die Summe der Spalten der Verkaufsmenge, des Umsatzes, der Kosten und des Profits berechnen
df_development = df_sales.groupby(['year', 'month']).agg({"amount" : "sum",
                                                          "revenue" : "sum",
                                                          "cost_total" : "sum",
                                                          "profit_total" : "sum"})

# Berechnung der Umsatzrendite
df_development['profit_margin'] = df_development['profit_total'] / df_development['revenue'] * 100

# Reset der Index-Spalte, um die Spalten "year" und "month" zu erhalten
df_development.reset_index(inplace=True)

df_development

In [ ]:
# Visualisierung der Entwicklung der Verkaufsmenge
fig, ax = plt.subplots(figsize=(10, 6))

# Daten plotten
ax.plot(df_development['year'].astype(str) + '-' + df_development['month'].astype(str), df_development['amount'])

# Grid hinzufuegen
ax.grid(color='lightgrey', linestyle='-', linewidth=0.5)

# Achsenbeschriftung und Titel hinzufuegen
ax.set_xlabel('Jahr-Monat', fontsize=12)
ax.set_ylabel('in Stück', fontsize=12)
ax.set_title('Entwicklung der Verkaufsmenge')

# Tausendertrennzeichen hinzufuegen
ax.get_yaxis().set_major_formatter(plt.FuncFormatter(lambda x, loc: "{:,}".format(int(x))))

# X-Achsenbeschriftung drehen
plt.xticks(rotation=45)

# Diagramm anzeigen
plt.tight_layout()
plt.show()

In [ ]:
# Visualisierung der Entwicklung des Umsatzes, der Kosten und des Profits
fig, ax = plt.subplots(figsize=(10, 6))

# Daten plotten
ax.plot(df_development['year'].astype(str) + '-' + df_development['month'].astype(str), df_development['revenue'])
ax.plot(df_development['year'].astype(str) + '-' + df_development['month'].astype(str), df_development['cost_total'])
ax.plot(df_development['year'].astype(str) + '-' + df_development['month'].astype(str), df_development['profit_total'])

# Grid hinzufuegen
ax.grid(color='lightgrey', linestyle='-', linewidth=0.5)

# Achsenbeschriftung und Titel hinzufuegen
ax.set_xlabel('Jahr-Monat', fontsize=12)
ax.set_ylabel('in €', fontsize=12)
ax.set_title('Entwicklung des Umsatzes, der Kosten und des Profits')

# Tausendertrennzeichen hinzufuegen
ax.get_yaxis().set_major_formatter(plt.FuncFormatter(lambda x, loc: "{:,}".format(int(x))))

# X-Achsenbeschriftung drehen
plt.xticks(rotation=45)

# Legende hinzufuegen
ax.legend(['Umsatz', 'Kosten', 'Profit'])

# Diagramm anzeigen
plt.tight_layout()
plt.show()

In [ ]:
# Visualisierung der Entwicklung der Umsatzrendite
fig, ax = plt.subplots(figsize=(10, 6))

# Daten plotten
ax.plot(df_development['year'].astype(str) + '-' + df_development['month'].astype(str), df_development['profit_margin'])

# Grid hinzufuegen
ax.grid(color='lightgrey', linestyle='-', linewidth=0.5)

# Achsenbeschriftung und Titel hinzufuegen
ax.set_xlabel('Jahr-Monat', fontsize=12)
ax.set_ylabel('in %', fontsize=12)
ax.set_title('Entwicklung der Umsatzrendite')

# X-Achsenbeschriftung drehen
plt.xticks(rotation=45)

# Diagramm anzeigen
plt.tight_layout()
plt.show()

## Prozentuelle Veränderung von Oktober 2021 bis November 2022

In [ ]:
# Veränderung vom Anfang bis zum Ende des Zeitraums in Prozent berechnen 

# Neues Dataframe mit erstellen (KPI, Change in %)
df_kpi = pd.DataFrame(columns=['kpi', 'change_in_percent'])

# Liste mit den KPIs erstellen
kpi = ['amount', 'revenue', 'cost_total', 'profit_total', 'profit_margin']

# Berechnung der Veränderung in Prozent
for i in kpi:
    change = (df_development[i].iloc[-1] / df_development[i].iloc[0] - 1) * 100
    df_kpi.loc[len(df_kpi)] = [i, change]

df_kpi

In [ ]:
# Visualisierung der Veränderung der KPIs als Säulendiagramm
fig, ax = plt.subplots(figsize=(10, 6))

# Daten plotten
bars = ax.bar(df_kpi['kpi'], df_kpi['change_in_percent'])

# Achsenbeschriftung und Titel hinzufuegen
ax.set_title('Veränderung der KPIs von Oktober 2021 bis September 2022')
ax.set_xlabel('KPI', fontsize=12)
ax.set_ylabel('in %', fontsize=12)

# Linie bei 0 hinzufuegen
ax.axhline(0, color='black', linewidth=0.5)

# Farbe der Säulen anpassen
ax.patches[0].set_color('green')
ax.patches[1].set_color('green')
ax.patches[2].set_color('red')
ax.patches[3].set_color('red')
ax.patches[4].set_color('red')

# Beschriftung der Säulen hinzufuegen
ax.bar_label(bars, fmt='%.2f%%', padding=2)

# X-Achsenbeschriftung auf Deutsch ("Verkaufsmenge", "Umsatz", "Kosten", "Profit", "Umsatzrendite")
ax.xaxis.set_ticks(np.arange(5))
ax.set_xticklabels(['Verkaufsmenge', 'Umsatz', 'Kosten', 'Profit', 'Umsatzrendite'])
plt.xticks(rotation=45)

# Diagramm anzeigen
plt.tight_layout()
plt.show()

## Stückbetrachtung

In [ ]:
# df_development kopieren
df_development_per_piece = df_development.copy()

# Berechnung des Umsatzes pro Stück
df_development_per_piece['revenue_per_piece'] = df_development_per_piece['revenue'] / df_development_per_piece['amount']    

# Berechnung der Kosten pro Stück
df_development_per_piece['cost_per_piece'] = df_development_per_piece['cost_total'] / df_development_per_piece['amount']

# Berechnung des Profits pro Stück
df_development_per_piece['profit_per_piece'] = df_development_per_piece['profit_total'] / df_development_per_piece['amount']

df_development_per_piece

In [ ]:
# Visualisierung der Entwicklung des Umsatzes pro Stück, der Kosten pro Stück und des Profits pro Stück
fig, ax = plt.subplots(figsize=(10, 6))

# Daten plotten
ax.plot(df_development_per_piece['year'].astype(str) + '-' + df_development_per_piece['month'].astype(str), df_development_per_piece['revenue_per_piece'])
ax.plot(df_development_per_piece['year'].astype(str) + '-' + df_development_per_piece['month'].astype(str), df_development_per_piece['cost_per_piece'])
ax.plot(df_development_per_piece['year'].astype(str) + '-' + df_development_per_piece['month'].astype(str), df_development_per_piece['profit_per_piece'])

# Grid hinzufuegen
ax.grid(color='lightgrey', linestyle='-', linewidth=0.5)

# Achsenbeschriftung und Titel hinzufuegen
ax.set_xlabel('Jahr-Monat', fontsize=12)
ax.set_ylabel('in €', fontsize=12)
ax.set_title('Entwicklung des Umsatzes pro Stück, der Kosten pro Stück und des Profits pro Stück')

# X-Achsenbeschriftung drehen
plt.xticks(rotation=45)

# Legende hinzufuegen
ax.legend(['Umsatz pro Stück', 'Kosten pro Stück', 'Profit pro Stück'])

# Diagramm anzeigen
plt.tight_layout()
plt.show()

In [ ]:
# Veränderung vom Anfang bis zum Ende des Zeitraums in Prozent berechnen 

# Neues Dataframe mit erstellen (KPI, Change in %)
df_kpi_per_piece = pd.DataFrame(columns=['kpi', 'change_in_percent'])

# Liste mit den KPIs erstellen
kpi = ['revenue_per_piece', 'cost_per_piece', 'profit_per_piece']

# Berechnung der Veränderung in Prozent
for i in kpi:
    change = (df_development_per_piece[i].iloc[-1] / df_development_per_piece[i].iloc[0] - 1) * 100
    df_kpi_per_piece.loc[len(df_kpi_per_piece)] = [i, change]

df_kpi_per_piece

In [ ]:
# Visualisierung der Veränderung der KPIs als Säulendiagramm
fig, ax = plt.subplots(figsize=(10, 6))

# Daten plotten
bars = ax.bar(df_kpi_per_piece['kpi'], df_kpi_per_piece['change_in_percent'])

# Achsenbeschriftung und Titel hinzufuegen
ax.set_title('Veränderung der KPIs von Oktober 2021 bis September 2022')
ax.set_xlabel('KPI', fontsize=12)
ax.set_ylabel('in %', fontsize=12)

# Linie bei 0 hinzufuegen
ax.axhline(0, color='black', linewidth=0.5)

# Farbe der Säulen anpassen
ax.patches[0].set_color('green')
ax.patches[1].set_color('red')
ax.patches[2].set_color('red')

# Beschriftung der Säulen hinzufuegen
ax.bar_label(bars, fmt='%.2f%%', padding=2)

# X-Achsenbeschriftung auf Deutsch ("Umsatz pro Stück", "Kosten pro Stück", "Profit pro Stück")
ax.xaxis.set_ticks(np.arange(3))
ax.set_xticklabels(['Umsatz pro Stück', 'Kosten pro Stück', 'Profit pro Stück'])
plt.xticks(rotation=45)

# Diagramm anzeigen
plt.tight_layout()
plt.show()

# 5) Detaillierte Analyse

## Hypothese 1: Preise haben sich verändert

In [ ]:
df_preise

In [ ]:
# Visualisierung der Entwicklung der Preise nach Produkte

# Breite der Balken festlegen
bar_width = 0.35

# Einzigartige Produkte und Jahre
products = df_preise['product'].unique()
years = df_preise['year'].unique()

# Farben für jedes Jahr
colors = ['tab:orange', 'tab:blue']

# Diagramm erstellen
fig, ax = plt.subplots()

# Balken für jedes Produkt und Jahr zeichnen
for i, product in enumerate(products):
    for j, year in enumerate(years):
        # Daten für das Produkt und Jahr filtern
        price = df_preise[(df_preise['product'] == product) & (df_preise['year'] == year)]['price'].values[0]
        # Balken zeichnen
        bar = ax.bar(i + j * bar_width, price, width=bar_width, label=f'{year}' if i == 0 else "", color=colors[j])
        # Preis über dem Balken anzeigen
        ax.text(bar[0].get_x() + bar[0].get_width() / 2, price + 10, f'{price}', ha='center')

# Achsenbeschriftungen
ax.set_xlabel('Produkt')
ax.set_ylabel('Preis in €')
ax.set_title('Preisentwicklung 2021-2022')

# X-Achsen-Ticks setzen
ax.set_xticks(np.arange(len(products)) + bar_width / 2)
ax.set_xticklabels(products)

# Legende hinzufügen
ax.legend(title='Jahr')

# Diagramm anzeigen
plt.tight_layout()
plt.show()


In [ ]:
# Grupperien des durchschnittliche Preises monatlich über beide Jahre hinweg
preis_pro_monat = df_sales.groupby(['year', 'month']).agg({"price" : "mean"})
preis_pro_monat.reset_index(inplace=True)
preis_pro_monat

In [ ]:
# Visualisierung der Entwicklung des durchschnittlichen Preises monatlich über beide Jahre hinweg
fig, ax = plt.subplots(figsize=(10, 6))

# Daten plotten
ax.plot(preis_pro_monat['year'].astype(str) + '-' + preis_pro_monat['month'].astype(str), preis_pro_monat['price'])

# Grid hinzufuegen
ax.grid(color='lightgrey', linestyle='-', linewidth=0.5)

# Achsenbeschriftung und Titel hinzufuegen
ax.set_xlabel('Jahr-Monat', fontsize=12)
ax.set_ylabel('in €', fontsize=12)
ax.set_title('Entwicklung des ø-Preises pro Transaktion')

# X-Achsenbeschriftung drehen
plt.xticks(rotation=45)

# Diagramm anzeigen
plt.tight_layout()
plt.show()

- Der durchschnittliche Preis je Transaktion ist gestiegen. Dies liegt daran, dass das teurere Tablet Pro sich im Laufe des Betrachtungszeitraum mehr verkauft hat als die anderen günstigeren Produkte.

## Hypothese 2: Menge an verkauften Produkten ist gesunken

In [ ]:
# Anzahl verkaufter Produkte pro Monat
menge_pro_monat = df_sales.groupby(['year', 'month']).agg({"amount" : "sum"})
menge_pro_monat.reset_index(inplace=True)
menge_pro_monat

In [ ]:
# Summe der Transaktionen pro Monat
transaktionen_pro_monat = df_sales.groupby(['year', 'month']).agg({"transaction_id" : "count"})
transaktionen_pro_monat.reset_index(inplace=True)
transaktionen_pro_monat

In [ ]:
# Beide als Barplot darstellen
fig, ax = plt.subplots(figsize=(10, 6))

# Daten plotten
ax.bar(menge_pro_monat['year'].astype(str) + '-' + menge_pro_monat['month'].astype(str), menge_pro_monat['amount'])
ax.bar(transaktionen_pro_monat['year'].astype(str) + '-' + transaktionen_pro_monat['month'].astype(str), transaktionen_pro_monat['transaction_id'])

# Achsenbeschriftung und Titel hinzufuegen
ax.set_xlabel('Jahr-Monat', fontsize=12)
ax.set_ylabel('Anzahl', fontsize=12)
ax.set_title('Anzahl verkaufter Produkte und Anzahl Transaktionen pro Monat')

# X-Achsenbeschriftung drehen
plt.xticks(rotation=45)

# Legende hinzufuegen
ax.legend(['Anzahl verkaufter Produkte', 'Anzahl Transaktionen'])

# Diagramm anzeigen
plt.tight_layout()
plt.show()

In [ ]:
# Durchschnittliche Anzahl der verkauften Produkte pro Transaktion
avg_menge_pro_monat = df_sales.groupby(['year', 'month']).agg({"amount" : "mean"})
avg_menge_pro_monat.reset_index(inplace=True)
avg_menge_pro_monat

In [ ]:
# Visualisierung der Entwicklung der durchschnittlichen Anzahl der verkauften Produkte pro Transaktion
fig, ax = plt.subplots(figsize=(10, 5))

# Daten plotten
ax.bar(avg_menge_pro_monat['year'].astype(str) + '-' + avg_menge_pro_monat['month'].astype(str), avg_menge_pro_monat['amount'])

# Achsenbeschriftung und Titel hinzufuegen
ax.set_xlabel('Jahr-Monat', fontsize=12)
ax.set_ylabel('Anzahl', fontsize=12)
ax.set_title('Durchschnittliche Anzahl der verkauften Produkte pro Transaktion')

# X-Achsenbeschriftung drehen
plt.xticks(rotation=45)

# Diagramm anzeigen
plt.tight_layout()
plt.show()

- Menge verkaufter Artikel und Anzahl Transaktionen sind gestiegen
- Durchschnittlich wurde je Transaktion aber etwas weniger gekauft

## Hypothese 3: Regionen haben unterschiedlich gut performt

In [ ]:
# Umsatz, Kosten und Gewinn pro Monat für Region A
df_sales_a = df_sales[df_sales['region'] == 'A'].groupby(pd.Grouper(key='date', freq='M')).agg({'amount': 'sum', 'revenue': 'sum', 'cost_total': 'sum', 'profit_total': 'sum'})

# Umsatz, Kosten und Gewinn pro Monat für Region B
df_sales_b = df_sales[df_sales['region'] == 'B'].groupby(pd.Grouper(key='date', freq='M')).agg({'amount': 'sum', 'revenue': 'sum', 'cost_total': 'sum', 'profit_total': 'sum'})

# Umsatz, Kosten und Gewinn pro Monat für Region C
df_sales_c = df_sales[df_sales['region'] == 'C'].groupby(pd.Grouper(key='date', freq='M')).agg({'amount': 'sum', 'revenue': 'sum', 'cost_total': 'sum', 'profit_total': 'sum'})

# Umsatz, Kosten und Gewinn pro Monat für Region D
df_sales_d = df_sales[df_sales['region'] == 'D'].groupby(pd.Grouper(key='date', freq='M')).agg({'amount': 'sum', 'revenue': 'sum', 'cost_total': 'sum', 'profit_total': 'sum'})

In [ ]:
# Umsatz, Kosten und Gewinn pro Monat für alle Regionen jeweils als Subplot visualisieren und alle Subplots zusammenfassen
fig, axes = plt.subplots(1, 4, figsize=(25, 5))
fig.suptitle('Profit, Cost and Revenue per month for all regions')

axes[0].plot(df_sales_a['amount'], label='A')
axes[0].plot(df_sales_b['amount'], label='B')
axes[0].plot(df_sales_c['amount'], label='C')
axes[0].plot(df_sales_d['amount'], label='D')
axes[0].set_title('amount sold')
axes[0].legend()

axes[1].plot(df_sales_a['revenue'], label='A')
axes[1].plot(df_sales_b['revenue'], label='B')
axes[1].plot(df_sales_c['revenue'], label='C')
axes[1].plot(df_sales_d['revenue'], label='D')
axes[1].set_title('Revenue')
axes[1].legend()

axes[2].plot(df_sales_a['cost_total'], label='A')
axes[2].plot(df_sales_b['cost_total'], label='B')
axes[2].plot(df_sales_c['cost_total'], label='C')
axes[2].plot(df_sales_d['cost_total'], label='D')
axes[2].set_title('Cost')

axes[3].plot(df_sales_a['profit_total'], label='A')
axes[3].plot(df_sales_b['profit_total'], label='B')
axes[3].plot(df_sales_c['profit_total'], label='C')
axes[3].plot(df_sales_d['profit_total'], label='D')
axes[3].set_title('Profit')

# y-Achse in Millionen formatieren
for ax in axes:
    ax.yaxis.set_major_formatter(
        plt.FuncFormatter(lambda x, loc: "{:,}".format(int(x))))

plt.tight_layout()

- Alle Regionen haben den gleichen Abwärtstrend beim Gewinn durchgemacht
    - Region C hatte die größte Gewinnsenkung
- Region D hat im Vergleich am besten performt

## Hypothese 4: Vertriebskanäle haben unterschiedlich performt

In [ ]:
# Umsatz, Kosten und Gewinn pro Woche für Verkaufskanal Online
df_sales_online = df_sales[df_sales['channel'] == 'online'].groupby(pd.Grouper(key='date', freq='M')).agg({'amount': 'sum', 'revenue': 'sum', 'cost_total': 'sum', 'profit_total': 'sum'})

# Umsatz, Kosten und Gewinn pro Woche für Verkaufskanal Phone
df_sales_phone = df_sales[df_sales['channel'] == 'phone'].groupby(pd.Grouper(key='date', freq='M')).agg({'amount': 'sum', 'revenue': 'sum', 'cost_total': 'sum', 'profit_total': 'sum'})

# Umsatz, Kosten und Gewinn pro Woche für Verkaufskanal Store
df_sales_store = df_sales[df_sales['channel'] == 'store'].groupby(pd.Grouper(key='date', freq='M')).agg({'amount': 'sum', 'revenue': 'sum', 'cost_total': 'sum', 'profit_total': 'sum'})

In [ ]:
# Umsatz, Kosten und Gewinn pro Woche für alle Verkaufskanäle jeweils als Subplot visualisieren und alle Subplots zusammenfassen
fig, axes = plt.subplots(1, 4, figsize=(25, 5))
fig.suptitle('Profit, Cost and Revenue per month for all channels')

axes[0].plot(df_sales_online['amount'], label='Online')
axes[0].plot(df_sales_phone['amount'], label='Phone')
axes[0].plot(df_sales_store['amount'], label='Store')
axes[0].set_title('amount sold')
axes[0].legend()

axes[1].plot(df_sales_online['revenue'], label='Online')
axes[1].plot(df_sales_phone['revenue'], label='Phone')
axes[1].plot(df_sales_store['revenue'], label='Store')
axes[1].set_title('Revenue')
axes[1].legend()

axes[2].plot(df_sales_online['cost_total'], label='Online')
axes[2].plot(df_sales_phone['cost_total'], label='Phone')
axes[2].plot(df_sales_store['cost_total'], label='Store')
axes[2].set_title('Cost')
axes[2].legend()

axes[3].plot(df_sales_online['profit_total'], label='Online')
axes[3].plot(df_sales_phone['profit_total'], label='Phone')
axes[3].plot(df_sales_store['profit_total'], label='Store')
axes[3].set_title('Profit')
axes[3].legend()

# y-Achse in Millionen formatieren
for ax in axes:
    ax.yaxis.set_major_formatter(
        plt.FuncFormatter(lambda x, loc: "{:,}".format(int(x))))
    
plt.tight_layout()

- Online hat den größten Gewinnverlust erfahren

### Vergleich der Online-Kanäle der Regionen A bis D

In [ ]:
# Anzahl verkaufter Produkte, Umsatz, Kosten und Gewinn pro Monat für die Regionen A, B, C und D in dem Kanal Online
df_sales_online_a = df_sales[(df_sales['region'] == 'A') & (df_sales['channel'] == 'online')].groupby(pd.Grouper(key='date', freq='M')).agg({"transaction_id": "count", 'amount': 'sum', 'revenue': 'sum', 'cost_total': 'sum', 'profit_total': 'sum'})

df_sales_online_b = df_sales[(df_sales['region'] == 'B') & (df_sales['channel'] == 'online')].groupby(pd.Grouper(key='date', freq='M')).agg({"transaction_id": "count", 'amount': 'sum', 'revenue': 'sum', 'cost_total': 'sum', 'profit_total': 'sum'})

df_sales_online_c = df_sales[(df_sales['region'] == 'C') & (df_sales['channel'] == 'online')].groupby(pd.Grouper(key='date', freq='M')).agg({"transaction_id": "count", 'amount': 'sum', 'revenue': 'sum', 'cost_total': 'sum', 'profit_total': 'sum'})

df_sales_online_d = df_sales[(df_sales['region'] == 'D') & (df_sales['channel'] == 'online')].groupby(pd.Grouper(key='date', freq='M')).agg({"transaction_id": "count", 'amount': 'sum', 'revenue': 'sum', 'cost_total': 'sum', 'profit_total': 'sum'})

In [ ]:
# Verkaufskanal Online für alle Regionen in einem Plot zusammenfassen
fig, axes = plt.subplots(1, 5, figsize=(25, 5))

axes[0].plot(df_sales_online_a['transaction_id'], label='A')
axes[0].plot(df_sales_online_b['transaction_id'], label='B')
axes[0].plot(df_sales_online_c['transaction_id'], label='C')
axes[0].plot(df_sales_online_d['transaction_id'], label='D')
axes[0].set_title('Number of transactions in Online Channel')
axes[0].legend()

axes[1].plot(df_sales_online_a['amount'], label='A')
axes[1].plot(df_sales_online_b['amount'], label='B')
axes[1].plot(df_sales_online_c['amount'], label='C')
axes[1].plot(df_sales_online_d['amount'], label='D')
axes[1].set_title('Amount sold in Online Channel')
axes[1].legend()

axes[2].plot(df_sales_online_a['revenue'], label='A')
axes[2].plot(df_sales_online_b['revenue'], label='B')
axes[2].plot(df_sales_online_c['revenue'], label='C')
axes[2].plot(df_sales_online_d['revenue'], label='D')
axes[2].set_title('Revenue in Online Channel')
axes[2].legend()

axes[3].plot(df_sales_online_a['cost_total'], label='A')
axes[3].plot(df_sales_online_b['cost_total'], label='B')
axes[3].plot(df_sales_online_c['cost_total'], label='C')
axes[3].plot(df_sales_online_d['cost_total'], label='D')
axes[3].set_title('Cost in Online Channel')
axes[3].legend()

axes[4].plot(df_sales_online_a['profit_total'], label='A')
axes[4].plot(df_sales_online_b['profit_total'], label='B')
axes[4].plot(df_sales_online_c['profit_total'], label='C')
axes[4].plot(df_sales_online_d['profit_total'], label='D')
axes[4].set_title('Profit in Online Channel')
axes[4].legend()

# y-Achse ohne wissenschaftliche Notation
for ax in axes:
    ax.ticklabel_format(style='plain', axis='y')
    
plt.tight_layout()

## Hypothese 5: Produkte haben unterschiedlich gut performt

In [ ]:
# Anzahl verkaufter Produkte, Umsatz, Kosten und Gewinn pro Monat für Produkt "Tablet Basic"
df_sales_basic = df_sales[df_sales['product'] == 'Tablet Basic'].groupby(pd.Grouper(key='date', freq='M')).agg({'amount': 'sum', 'revenue': 'sum', 'cost_total': 'sum', 'profit_total': 'sum'})

# Anzahl verkaufter Produkte, Umsatz, Kosten und Gewinn pro Monat für Produkt "Tablet Mini"
df_sales_mini = df_sales[df_sales['product'] == 'Tablet Mini'].groupby(pd.Grouper(key='date', freq='M')).agg({'amount': 'sum', 'revenue': 'sum', 'cost_total': 'sum', 'profit_total': 'sum'})

# Anzahl verkaufter Produkte, Umsatz, Kosten und Gewinn pro Monat für Produkt "Tablet Pro"
df_sales_pro = df_sales[df_sales['product'] == 'Tablet Pro'].groupby(pd.Grouper(key='date', freq='M')).agg({'amount': 'sum', 'revenue': 'sum', 'cost_total': 'sum', 'profit_total': 'sum'})

In [ ]:
# Umsatz, Kosten und Gewinn pro Monat für alle Produkte jeweils als Subplot visualisieren und alle Subplots zusammenfassen
fig, axes = plt.subplots(1, 4, figsize=(25, 5))
fig.suptitle('Amount sold, Profit, Cost and Revenue per month for all products')

axes[0].plot(df_sales_basic['amount'], label='Basic')
axes[0].plot(df_sales_mini['amount'], label='Mini')
axes[0].plot(df_sales_pro['amount'], label='Pro')
axes[0].set_title('Amount sold')
axes[0].legend()

axes[1].plot(df_sales_basic['revenue'], label='Basic')
axes[1].plot(df_sales_mini['revenue'], label='Mini')
axes[1].plot(df_sales_pro['revenue'], label='Pro')
axes[1].set_title('Revenue')
axes[1].legend()

axes[2].plot(df_sales_basic['cost_total'], label='Basic')
axes[2].plot(df_sales_mini['cost_total'], label='Mini')
axes[2].plot(df_sales_pro['cost_total'], label='Pro')
axes[2].set_title('Cost')
axes[2].legend()

axes[3].plot(df_sales_basic['profit_total'], label='Basic')
axes[3].plot(df_sales_mini['profit_total'], label='Mini')
axes[3].plot(df_sales_pro['profit_total'], label='Pro')
axes[3].set_title('Profit')
axes[3].legend()

# y-Achse in Millionen formatieren
for ax in axes:
    ax.yaxis.set_major_formatter(
        plt.FuncFormatter(lambda x, loc: "{:,}".format(int(x))))

plt.tight_layout()

In [ ]:
# Veränderung von Anfang bis Ende für die Produkte "Tablet Basic", "Tablet Mini" und "Tablet Pro"
def calc_change(df, productname):
    print("Veränderung Anzahl sold", productname, ":", (df.iloc[-1]['amount'] - df.iloc[0]['amount']) / df.iloc[0]['amount'] * 100)
    print("Veränderung Umsatz", productname, ":", (df.iloc[-1]['revenue'] - df.iloc[0]['revenue']) / df.iloc[0]['revenue'] * 100)
    print("Veränderung Kosten", productname, ":", (df.iloc[-1]['cost_total'] - df.iloc[0]['cost_total']) / df.iloc[0]['cost_total'] * 100)
    print("Veränderung Gewinn", productname, ":", (df.iloc[-1]['profit_total'] - df.iloc[0]['profit_total']) / df.iloc[0]['profit_total'] * 100)
    print()


calc_change(df_sales_basic, "Tablet Basic")
calc_change(df_sales_mini, "Tablet Mini")
calc_change(df_sales_pro, "Tablet Pro")

- Tablet Basic hat sich stark verschlechtert -> Von ca. 9,5 Mio. Gewinn zu 50.000
    - Wurde auch deutlich weniger verkauft
- Gewinn von Tablet Mini hat sich zum Jahreswechsel verschlechtert, aber hat sich danach verbessert und nun ist das profitabelste Produkt
    - Anzahl verkaufter Produkte stagniert, aber leicht gestiegen
- Gewinn von Tablet Pro hat sich stark verbessert, aber Trend ist zum Ende 2022 negativ
    - Anzahl, Umsatz und Kosten sind sehr stark gestiegen
    - Jedoch sind Kosten stärker gestiegen als Umsatz

### Transaktionen mit Verlust

In [ ]:
# Transaktionen mit Verlust
df_sales_negative_profit = df_sales[df_sales['profit_total'] < 0]
df_sales_negative_profit

In [ ]:
# Welche Produkte wurden mit einem Verlust verkauft?
df_sales_negative_profit['product'].unique()

- Nur mit Tablet Basic wurden Verlustgeschäfte gemacht, insg. 2314 Transaktionen

In [ ]:
# Summe des Verlusts
df_sales_negative_profit['profit_total'].sum()

- Mit dem Tablet Basic wurde in 2314 Transaktionen insg. 460 Tsd. EUR Verlust gemacht 

In [ ]:
# In welchen Verkaufskanälen und Regionen wurden die Produkte mit negativem Gewinn verkauft und wie oft?
df_sales_negative_profit.groupby(['region', 'channel']).agg({'transaction_id': 'count', 'profit_total': 'sum'})

- Meistens in den Stores

### Wie oft wurden die Produkte in den einzelnen Regionen verkauft?

In [ ]:
# Group by channel, product and sum up amount
sales_per_product_by_channel = df_sales.groupby(['channel', 'product']).agg({'amount': 'sum'})
sales_per_product_by_channel = sales_per_product_by_channel.reset_index()
sales_per_product_by_channel

In [ ]:
plt.figure(figsize=(10, 6))

sns.barplot(data=sales_per_product_by_channel, x='channel', y='amount', hue='product')

plt.title('Amount sold per product, grouped by channel')
plt.xlabel('Channel')
plt.ylabel('Amount sold')
plt.show()

### Wie oft wurden die Produkte in den einzelnen Regionen verkauft?

In [ ]:
# Group by region, product and sum up amount
sales_per_product_by_region = df_sales.groupby(['region', 'product']).agg({'amount': 'sum'})
sales_per_product_by_region = sales_per_product_by_region.reset_index()
sales_per_product_by_region

In [ ]:
plt.figure(figsize=(10, 6))

sns.barplot(data=sales_per_product_by_region, x='region', y='amount', hue='product')

plt.title('Amount sold per product, grouped by region')
plt.xlabel('Region')
plt.ylabel('Amount sold')
plt.show()

## Hypothese 7: Es wurden mehr Rabatte gewährt

In [ ]:
# Durchschnittlicher Rabatt pro Monat
rabatt_pro_monat = df_sales.groupby(pd.Grouper(key='date', freq='M')).agg({'discount': 'mean'})
rabatt_pro_monat

In [ ]:
# Als barplot visualisieren
plt.figure(figsize=(10, 6))

sns.barplot(data=rabatt_pro_monat, x=rabatt_pro_monat.index, y='discount')
plt.title('Average discount per month')
plt.xlabel('Month')
plt.ylabel('Avg Discount')

#plt.ylim(0.90, 1.0)

plt.xticks(rotation=45)

plt.show()

- Nein, Rabatte haben sich sogar verringert

In [ ]:
# Durchschnittliche Rabatte pro Produkt pro Monat
rabatt_pro_monat_pro_produkt = df_sales.groupby(['product', 'year', "month"]).agg({'discount': 'mean'})  
rabatt_pro_monat_pro_produkt

In [ ]:
# Entfernen der Multi-Index-Spalten
rabatt_pro_monat_pro_produkt = rabatt_pro_monat_pro_produkt.reset_index()

plt.figure(figsize=(10, 6))

sns.lineplot(data=rabatt_pro_monat_pro_produkt, x='month', y='discount', hue='product')
plt.title('Average discount per month per product')
plt.xlabel('Month')

## Hypothese 8: Kundenzufriedenheit hat sich verändert

In [ ]:
# Durschnittliches Customer Rating pro Monat für Tablet Basic
rating_basic = df_sales[df_sales['product'] == 'Tablet Basic'].groupby(pd.Grouper(key='date', freq='M')).agg({'customer_rating': 'mean'})

# Durschnittliches Customer Rating pro Monat für Tablet Mini
rating_mini = df_sales[df_sales['product'] == 'Tablet Mini'].groupby(pd.Grouper(key='date', freq='M')).agg({'customer_rating': 'mean'})

# Durschnittliches Customer Rating pro Monat für Tablet Pro
rating_pro = df_sales[df_sales['product'] == 'Tablet Pro'].groupby(pd.Grouper(key='date', freq='M')).agg({'customer_rating': 'mean'})

In [ ]:
sns.lineplot(data=rating_basic, x=rating_basic.index, y='customer_rating', label='Basic')
sns.lineplot(data=rating_mini, x=rating_mini.index, y='customer_rating', label='Mini')
sns.lineplot(data=rating_pro, x=rating_pro.index, y='customer_rating', label='Pro')

plt.title('Average customer rating per month')
plt.xlabel('Month')
plt.ylabel('Avg Customer Rating')

plt.ylim(0, 10)
plt.xticks(rotation=45)

plt.show()

- Kundenzufriedenheit für Tablet Basic ist relativ stark gesunken
- Kundenzufriedenheit für Tablet Mini ist im Vergleich leichter gesunken
- Kundenzufriedenheit für Tablet Pro ist gestiegen

### (Versuch, Umsatz nach Customer Rating als Regression zu plotten)

In [ ]:
umsatz_by_customer_rating = df_sales.groupby('customer_rating').agg({'revenue': 'mean',
                                                                     "customer_rating": "mean"})

sns.regplot(x="customer_rating", y="revenue", data=umsatz_by_customer_rating, ci=True, 
            line_kws={"color": "red"}, scatter_kws={"alpha": 0.5})

In [ ]:
# Lineare Regression für die Spalte "revenue" erstellen
model = smf.ols(formula='revenue ~ customer_rating', data=df_sales).fit()
model.summary2()

### Umsatz je Customer Rating für jedes Produkt

In [ ]:
sales_by_rating = df_sales.groupby(["date", 'product']).agg({'customer_rating': 'mean',
                                                            'revenue': 'sum'})

plt.figure(figsize=(10, 6))
sns.scatterplot(data=sales_by_rating, x='customer_rating', y='revenue', hue='product')
plt.title('Customer Rating and Revenue per product')
plt.xlabel('Customer Rating')
plt.ylabel('Revenue')
plt.legend(title="Product")

# y-Achse in Millionen formatieren
plt.gca().yaxis.set_major_formatter(
    plt.FuncFormatter(lambda x, loc: "{:,}".format(int(x))))

plt.show()

In [ ]:
correlation_sales_rating = sales_by_rating.corr()
correlation_sales_rating[['customer_rating']].loc[['revenue']]

- Positive Korrelation: Es gibt eine positive Korrelation zwischen den Kundenbewertungen und den Verkaufszahlen. Dies deutet darauf hin, dass Produkte mit höheren Kundenbewertungen tendenziell höhere Verkaufszahlen aufweisen.

In [ ]:
sales_by_rating = sales_by_rating.reset_index()
sns.lmplot(data=sales_by_rating, x="customer_rating", y="revenue", hue='product', ci=None, height=6, aspect=1.5, line_kws={"color": "red"}, scatter_kws={"alpha": 0.5})
plt.title('Customer Rating and Revenue per product')
plt.xlabel('Customer Rating')
plt.ylabel('Revenue')
plt.legend(title="Product")

# y-Achse in Millionen formatieren
plt.gca().yaxis.set_major_formatter(
    plt.FuncFormatter(lambda x, loc: "{:,}".format(int(x))))

plt.show()

## Hypothese 9: Downtime war ein Grund für die schlechte Entwicklung des Online-Stores

In [ ]:
df_downtime

In [ ]:
# Alle Transaktionen im Kanal Online in einem Dataframe zusammenfassen
sales_online = df_sales[df_sales['channel'] == 'online']
sales_online

In [ ]:
# # Merge der Downtime-Daten in die Transaktionsdaten
sales_online = pd.merge(sales_online, df_downtime, how='left', on=['year', 'region'])

sales_online

In [ ]:
# Transaktionen nach regionen
sales_online_a = sales_online[sales_online['region'] == 'A']
sales_online_b = sales_online[sales_online['region'] == 'B']
sales_online_c = sales_online[sales_online['region'] == 'C']
sales_online_d = sales_online[sales_online['region'] == 'D']

# Umsätze monatlich aggregieren
monthly_sales_online_a = sales_online_a.groupby(pd.Grouper(key='date', freq='M')).agg({'transaction_id': 'count'})
monthly_sales_online_b = sales_online_b.groupby(pd.Grouper(key='date', freq='M')).agg({'transaction_id': 'count'})
monthly_sales_online_c = sales_online_c.groupby(pd.Grouper(key='date', freq='M')).agg({'transaction_id': 'count'})
monthly_sales_online_d = sales_online_d.groupby(pd.Grouper(key='date', freq='M')).agg({'transaction_id': 'count'})

In [ ]:
def downtime_calculations (df, region):
    # Spalte date, year und month hinzufügen
    df = df.assign(date=df.index,
                   year=df.index.year,
                   month=df.index.month)
    
    # Spalte region hinzufügen
    df = df.assign(region=region)
    
    # Spalte downtime hinzufügen
    df = pd.merge(df, df_downtime, how='left', on=['year', 'region'])

    # Spalte possible_amount, delta und delta_rel hinzufügen
    df['possible_transactions'] = df['transaction_id'] / (1 - df['downtime'])
    df['delta'] = df['possible_transactions'] - df['transaction_id']
    df['delta_rel'] = df['delta'] / df['transaction_id']

    # Spalte transaction_id in "transaction_count" umbenennen
    df = df.rename(columns={'transaction_id': 'transaction_count'})
    
    return df

monthly_sales_online_a = downtime_calculations(monthly_sales_online_a, "A")
monthly_sales_online_b = downtime_calculations(monthly_sales_online_b, "B")
monthly_sales_online_c = downtime_calculations(monthly_sales_online_c, "C")
monthly_sales_online_d = downtime_calculations(monthly_sales_online_d, "D")
    

In [ ]:
monthly_sales_online_a

In [ ]:
monthly_sales_online_b

In [ ]:
monthly_sales_online_c

In [ ]:
monthly_sales_online_d

In [ ]:
# Alle Deltas der vier Regionen plotten
plt.figure(figsize=(10, 6))
sns.lineplot(data=monthly_sales_online_a, x='date', y='delta', label='A')
sns.lineplot(data=monthly_sales_online_b, x='date', y='delta', label='B')
sns.lineplot(data=monthly_sales_online_c, x='date', y='delta', label='C')
sns.lineplot(data=monthly_sales_online_d, x='date', y='delta', label='D')

### Prognose erstellen:
- Anhand der Monate November/Dezember 2021 eine Prognose für Januar 2022 machen und schauen, ob die Abweichung in der Prognose vom realen Wert durch die veränderte Downtime erklärt wird 

#### Beispielhaft für Region A:

In [ ]:
# Anzahl Transaktionen pro Tag berechnen
daily_sales_online_a = sales_online_a.groupby(pd.Grouper(key='date', freq='D')).agg({'transaction_id': 'count'})
daily_sales_online_a

In [ ]:
# Transaktionen pro Tag für die Monate November 2021 bis Januar 2022
daily_sales_online_a = daily_sales_online_a.loc['2021-11-01':'2022-01-31']

# In Dataframe umwandeln
daily_sales_online_a = daily_sales_online_a.reset_index()
daily_sales_online_a

In [ ]:
# Exponentielle Glättung mit alpha=0.2
daily_sales_online_a['ewm'] = daily_sales_online_a['transaction_id'].ewm(alpha=0.2, adjust=False).mean()
daily_sales_online_a

In [ ]:
# Forecast Error berechnen
daily_sales_online_a['forecast_error'] = ((daily_sales_online_a['transaction_id'] - daily_sales_online_a['ewm']) / daily_sales_online_a['transaction_id']) * 100

# Alle Zeilen anzeigen
pd.set_option('display.max_rows', None)
daily_sales_online_a

In [ ]:
# Group by week
weekly_sales_online_a = daily_sales_online_a.groupby(pd.Grouper(key='date', freq='W')).agg({'transaction_id': 'sum', 'ewm': 'sum'})
weekly_sales_online_a

In [ ]:
# Kalkuliere Prognosefehler als Prozent
weekly_sales_online_a['forecast_error'] = ((weekly_sales_online_a['transaction_id'] - weekly_sales_online_a['ewm']) / weekly_sales_online_a['transaction_id']) * 100
weekly_sales_online_a